## **Explain the predictions of the Satellite Image Classification Model**

**Author-1 : Purushothaman Natarajan**

**Author-2 : Kamal Basha**

Source Dataset : Kaggle

Dataset Name : AID: A scene classification dataset

Link : https://www.kaggle.com/datasets/jiayuanchengala/aid-scene-classification-datasets

Approach: The developed model is explained using the explainers namely LIME and GradCAMM. Both these explainers are post-hoc in nature, so they explain the predictions from model using the input and the output, hence it does not demand the training dataset.

LIME: It perturbs the input and fits it in a linear model to verify whether the selected feature is contributing for the prediction or not.

GradCAMM: It uses class activation maps, where in the gradients with high score is visualized; this method suitable for model based on CNN networks.

Atlast, we also integrate the model and the explainer on a interface (Using Gradio) through which the user can upload a image and get its class label along with its explantion.

In [1]:
!pip install gradio
!pip install tensorflow
!pip install keras
!pip install lime
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load the Model

import os
import tensorflow as tf

model_path = "/content/drive/MyDrive/RS LLM/Classification Model/Model and Data Backups/Model/11 Class Model/try_1_InceptionResNetV2_model.h5"

# Load the model without custom objects
loaded_model = tf.keras.models.load_model(model_path)

# Print the loaded model names
print("Loaded model")

Loaded model


In [6]:
# Load the Encoder-Decoder

import numpy as np
from sklearn.preprocessing import LabelEncoder

master_folder = "/content/drive/MyDrive/RS LLM/Classification Model/Model and Data Backups"

# Load label encoder classes
label_encoder_classes = np.load(os.path.join(master_folder, 'onehot_encoded_labels.npy'))

# Set the classes_ attribute of the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.classes_ = label_encoder_classes

class_labels = ['Airport', 'BareLand', 'BaseballField', 'Beach', 'Bridge', 'Center', 'Church', 'Commercial', 'DenseResidential', 'Desert', 'Farmland']
label_encoder.fit(class_labels)

print('Encoder-Decoder Loaded')

Encoder-Decoder Loaded


# **Grad-CAM**

In [7]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0][0

In [8]:
# last convolution layer for grad-camm explanation

last_conv_layer_name = "conv_7b_ac"

In [9]:
## GRAD-CAM

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        inputs = model.inputs, outputs = [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        inputs = tf.cast(img_array, tf.float32)
        last_conv_layer_output, preds = grad_model(img_array)
        class_channel = preds[:, pred_index]
        # if pred_index is None:
        #     pred_index = tf.argmax(preds[0])
        # class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# **SP-LIME**

In [10]:
# Initialize the LIME Explainer

import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from lime.lime_image import LimeImageExplainer, SegmentationAlgorithm
import numpy as np
import gradio as gr
import matplotlib as mpl

# Define the target size for resizing
target_size = (299, 299)

# Function to generate SP-LIME mask and overlay on the image for top N predictions
def generate_splime_mask_top_n(img_array, model, explainer, top_n=1, num_features=100, num_samples=300):
    # Use superpixel segmentation for SP-LIME
    segmentation_fn = SegmentationAlgorithm('quickshift', kernel_size=4, max_dist=200, ratio=0.2)

    explanation_instance = explainer.explain_instance(
        img_array, model.predict, top_labels=top_n, hide_color=0,
        num_samples=num_samples, num_features=num_features, segmentation_fn=segmentation_fn
    )
    explanation_mask = explanation_instance.get_image_and_mask(
        explanation_instance.top_labels[0], positive_only=True,
        num_features=num_features, hide_rest=True
    )

    return explanation_mask

# Define different parameters for LimeImageExplainer
segmentation_algorithms = ['quickshift', 'slic']
kernel_sizes = [2]
max_dists = [100]
ratios = [0.1]
num_features_values = [100]
num_samples_values = [300]

In [11]:
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from lime.lime_image import LimeImageExplainer, SegmentationAlgorithm
import os

# path to save the files in the folder
output_folder = "/content/drive/MyDrive/RS LLM/Classification Model/Explainer outputs"
os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

# Counter for images
image_counter = 0


def save_and_display_gradcam(array, heatmap, cam_path="cam.jpg", alpha=0.8):
    img = array
    heatmap = np.uint8(255 * heatmap)
    jet = plt.cm.jet
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)
    superimposed_img.save(cam_path)
    return superimposed_img

def classify_image_and_explain(image, num_samples=10, num_features=10):
    global image_counter
    image_counter += 1

    target_size = (299, 299)
    image = image.resize(target_size)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    array = img_to_array(image)
    img_array = array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    preds = loaded_model.predict(img_array)
    top_prediction = np.argmax(preds[0])
    top_label = label_encoder.inverse_transform([top_prediction])[0]
    top_prob = preds[0][top_prediction]

    def top_3_predictions(predictions):
        top_3_indices = np.argsort(predictions)[-3:][::-1]
        top_3_predictions = predictions[top_3_indices]
        return top_3_predictions

    top_3_predictions = top_3_predictions(preds[0])
    top_3_labels = label_encoder.inverse_transform(np.argsort(preds[0])[-3:][::-1])

    explainer = LimeImageExplainer()
    segmentation_fn = SegmentationAlgorithm('quickshift', kernel_size=2, max_dist=100, ratio=0.1)
    splime_mask, explanation_instance = generate_splime_mask_top_n(
        img_array[0], loaded_model, explainer, top_n=3, num_features=num_features, num_samples=num_samples
    )

    loaded_model.layers[-1].activation = None
    output_tensor = loaded_model.output[0]
    heatmap = make_gradcam_heatmap(img_array, loaded_model, last_conv_layer_name)
    grad_cam_explanation = save_and_display_gradcam(array, heatmap)
    # Convert the SPLIME mask to PIL image
    splime_mask_pil = keras.utils.array_to_img(splime_mask)

    # Save input image
    input_img_path = os.path.join(output_folder, f"{top_label}_input_{image_counter}.jpg")
    image.save(input_img_path)
    # Save LIME explanation image
    lime_img_path = os.path.join(output_folder, f"{top_label}_lime_explanation_{image_counter}.jpg")
    splime_mask_pil.save(lime_img_path)
    # Save Grad-CAM explanation image
    gradcam_img_path = os.path.join(output_folder, f"{top_label}_gradcam_explanation_{image_counter}.jpg")
    grad_cam_explanation.save(gradcam_img_path)

    return splime_mask_pil, grad_cam_explanation, top_label

In [ ]:
explainer_interface = gr.Interface(
    fn=classify_image_and_explain,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(10, 100, value=10, step=1, label="Number of Samples", info="Choose between 10 and 100"),
	      gr.Slider(10, 1000, value=10, step=10, label="Number of Features", info="Choose between 10 and 1000")
       ],
    outputs=[gr.Image(label="LIME Explanation"), gr.Image(label="GradCAM Explanation"), "text"],
    title="Image Classification with LIME & GRAD-CAM explanation",
    description="Upload an image to classify and explain it using SP-LIME and Grad-CAM."
)


if __name__ == "__main__":
    explainer_interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cc20073e7b9c1e3141.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 7s 7s/step


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 1s 636ms/step


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 1s 747ms/step


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step
